In [1]:
import requests
import time
import re
import openpyxl
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import date

In [2]:
GLOBAL = 'https://www.gipernn.ru'
LINK = 'https://www.gipernn.ru/arenda-ofisov?per-page=50'

Добыча колличества ссылок

In [3]:
html = requests.get(LINK)
soup = BeautifulSoup(html.text, 'html.parser')
count = soup.find('div', class_='count')

count = int(count.text.split(':')[1].strip())
page_count = int(np.ceil(count / 50))

print(count)

345


Составляем массив ссылок

In [4]:
links = []
for page in tqdm(range(page_count + 1), ascii=True):
    html = requests.get(f"{LINK}&page={page}")
    soup = BeautifulSoup(html.text, 'html.parser')
    trs = soup.find('tbody').find_all('tr')
    for tr in trs:
        link = tr.find('a')
        if link:
            link = link.get('href')
            link = GLOBAL + link
            links.append(link)
    links = list(set(links))
print(len(links))   


100%|##########| 8/8 [00:03<00:00,  2.63it/s]

317


Составление своей базы данных

In [5]:
df = pd.DataFrame()
for link in tqdm(links, ascii=True):
    html = requests.get(link)
    soup = BeautifulSoup(html.text, 'html.parser')

    tmp = {} # словарь для временных данных
    tmp['Ссылка'] = link

    header = soup.find('h1')
    if header:
        header = header.text.strip().replace('\xa0', ' ')
    tmp['Заголовок'] = header

    price = soup.find('div', class_ = 'price')
    if price:
        price = price.text.replace('\xa0', ' ').strip().split('  ')
    tmp['Цена'] = price

    description = soup.find('div', class_='formatted-text')
    if description:
        description = description.text.strip().replace('\xa0', ' ')
    tmp['Описание'] = description
    
    try:
        w0 = soup.find('table', id = 'w0')
        if w0:
            w0 = w0.find_all('tr')
            for tr in w0:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                tmp[key] = val
    except:
        print('w0 not found')
 
    try:
        w2 = soup.find('table', id = 'w2')
        if w2:
            w2 = w2.find_all('tr')
            for tr in w2:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('w2 not found')

    try:
        properties = soup.find('div', class_ = 'properties m-t-2')
        if properties:
            properties = properties.find_all('div', class_ = 'property')
            for property in properties:
                key = property.find('div', class_ = 'property-name').text.strip()
                val = property.find('div', class_ = 'property-value').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('properties m-t-2 not found')
    
    try:
        buildings = soup.find('div', 'around').find_all('div', 'group')
        for building in buildings:
            name = building.find('div','group-body').find('span').text
            dist = building.find('div','group-body').find('div', 'company').find('span', 'company__distance').text
            tmp[name] = dist   
    except:
        pass

    df = pd.concat([df, pd.DataFrame(tmp)], ignore_index=True)

df['Источник'] = 'gipernn'
df['Сегмент'] = 'Комерческая недвижимость'
df['Подсегмент'] = 'Аренда офисов'
df['Город'] = 'Нижний Новгород'
df['Дата парсинга'] = str(date.today())

100%|##########| 317/317 [01:23<00:00,  3.80it/s]


Посмотрим на результаты вкрадце

In [6]:
df.head()

,Ссылка,Заголовок,Цена,Описание,№,Район,Адрес,Площадь,Этаж / этажность,Высота потолков,...,Торг уместен,Этажность,Материал стен,Площадь земельного участка,Общая площадь зданий,Источник,Сегмент,Подсегмент,Город,Дата парсинга
0,https://www.gipernn.ru/arenda-pomescheniy/ul-k...,"Аренда помещения на ул. Куйбышева, д. 57Москов...",20 000 руб./мес.,Сдам нежилое помещение на улице Куйбышева. ...,2864385,Московский,"ул. Куйбышева, д. 57",29 м²,1 / 9,Уточнить,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда офисов,Нижний Новгород,2022-12-21
1,https://www.gipernn.ru/arenda-pomescheniy/ul-s...,"Аренда помещения на ул. Салганская, д. 24Совет...",614 240 руб./мес.,"Сдаётся в аренду отдельный офисный блок, общей...",2854594,Советский,"ул. Салганская, д. 24БЦ Салганская улица, 24",880 м²,2 / 5,3 м,...,да,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда офисов,Нижний Новгород,2022-12-21
2,https://www.gipernn.ru/arenda-pomescheniy/ul-g...,"Аренда помещения на ул. Героя Рябцева, д. 27Мо...",4 000 руб./мес.,Сдам рабочее место ~5 м.кв. в помещении свобод...,2801485,Московский,"ул. Героя Рябцева, д. 27",5 м²,1 / 4,"2,8 м",...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда офисов,Нижний Новгород,2022-12-21
3,https://www.gipernn.ru/arenda-pomescheniy/ul-s...,"Аренда помещения на ул. Свободы, д. 15Сормовск...",140 140 руб./мес.,"Аренда офисного помещения в современном БЦ ""Со...",2801471,Сормовский,"ул. Свободы, д. 15БЦ Союзный","215,6 м²",6 / 9,3 м,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда офисов,Нижний Новгород,2022-12-21
4,https://www.gipernn.ru/arenda-pomescheniy/ul-v...,"Аренда помещения на ул. Верхнепечёрская, д. 14...",11 000 руб./мес.,Торгово-офисная площадь в ТЦ ВерхнеПечерский.\...,2835446,Нижегородский,"ул. Верхнепечёрская, д. 14бТЦ ВерхнеПечерский",11 м²,цоколь / 2,3 м,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда офисов,Нижний Новгород,2022-12-21


Экспортируем таблицу

In [7]:
df.to_excel('ofice-rent.xlsx', index=False)